In [21]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [3]:
pred_df = pd.read_pickle('./output_file/20230315_multi_factor_v2.0.0_evaluation.pickle')

In [36]:
def buy_plan(predicted_test, close_col='Close',pred_col='pred_Close',input_money=100, lever=1, handle_fee = 0.0005, trade_percent_cutoff=0.0, print_history=False, seq_len=128):
    ############### args ##########
    # input = predictied dataframe with 'close_original' and 'pred_close' columns
    # input_money = trading money
    # lever = 槓桿
    # handle_fee = trading fee
    # trade_percent_cutoff = if expected income is lower that cutoff, skip this trade
    # print_history = print out each trading detail
    ############## output ##########
    # df = with 'total_earn_money', 'day_earn_percent', 'total_day_earn_percent' columns
    # win_rate = trading win percent
    ###############################
    
    # input 128 datas

    input_df = predicted_test[seq_len:].reset_index(drop=True).copy()
    total_earn_money = [input_money]
    total_day_earn_percent = [0]
    trading_list = []
    day = []
    win_count = 0
    trade = ''
    for i in range(len(input_df)-1):
        earn_persent          = (input_df[close_col][i+1] - input_df[close_col][i])/input_df[close_col][i]
        expected_earn_percent = (input_df[pred_col][i+1] - input_df[close_col][i])    /input_df[close_col][i]
        charge_handle_fee     = handle_fee*input_money*lever
        trading_winlost       = np.sign(earn_persent) * np.sign(expected_earn_percent)
        trading_strategy      = 'Long' if expected_earn_percent > 0 else 'Short' 
        if abs(expected_earn_percent*100) < trade_percent_cutoff:
            total_earn_money.append(input_money)
            continue
        day_earn_percent = (abs(earn_persent)*lever*trading_winlost) + 1
        earn_money = input_money*day_earn_percent
        if trade != trading_strategy:
            earn_money -= charge_handle_fee
            trade = trading_strategy
            
        if print_history == True:
            print(f'Trading {i+1}')
            print(f'Trade_{trade}----:', f'{(earn_persent*lever)*100}%')
            print('handle_fee---:', charge_handle_fee)
            print('input_money--:',input_money)
            print('output_money-:',earn_money)
            print('--------------------------')
        total_earn_money.append(earn_money)
        trading_list.append(trading_strategy)
        #total_day_earn_percent.append((1-day_earn_percent)*100)
        day.append(i)
        if day_earn_percent > 1:
            win_count += 1
        input_money = earn_money
        if earn_money < 0:
            print('You Are broken XD')
            return input_df
    input_df['trading'][1:] = trading_list
    input_df['total_earn_money']       = total_earn_money
    input_df['day_earn_percent']       = (input_df['total_earn_money'].shift(0) - input_df['total_earn_money'].shift(1))/input_df['total_earn_money'].shift(0)*100
    input_df['total_day_earn_percent'] = input_df['total_earn_money'].apply(lambda x: (x- total_earn_money[0])/total_earn_money[0]*100)
    win_rate = (win_count/len(day))
    input_df = input_df.fillna('-')
    print('trading win rate:', win_rate)
    return input_df

In [37]:
input_df= buy_plan(pred_df, input_money=10000, lever=2, trade_percent_cutoff=0.0, print_history=True)

Trading 1
Trade_Short----: -4.519225209598141%
handle_fee---: 10.0
input_money--: 10000
output_money-: 10441.922520959813
--------------------------
Trading 2
Trade_Short----: -0.4661563398445529%
handle_fee---: 10.441922520959814
input_money--: 10441.922520959813
output_money-: 10490.598204792923
--------------------------
Trading 3
Trade_Short----: -0.43759783691473153%
handle_fee---: 10.490598204792922
input_money--: 10490.598204792923
output_money-: 10536.504835616512
--------------------------
Trading 4
Trade_Short----: 0.8497792356740888%
handle_fee---: 10.536504835616512
input_money--: 10536.504835616512
output_money-: 10446.967805357646
--------------------------
Trading 5
Trade_Short----: -0.8083127211617325%
handle_fee---: 10.446967805357646
input_money--: 10446.967805357646
output_money-: 10531.411975104023
--------------------------
Trading 6
Trade_Short----: 0.1366517934062539%
handle_fee---: 10.531411975104024
input_money--: 10531.411975104023
output_money-: 10517.0206117

KeyError: 'trading'

In [17]:
input_df

,open_MA_ptc_10,High_MA_ptc_10,Low_MA_ptc_10,Close_MA_ptc_10,Bias,Cmo_10,Cci_10,ZhangDieFu,Rsi_10,Ic_10,...,Cci_200,Rsi_200,Ic_200,pred_Close_MA_ptc_10,Close,Close_MA_10,pred_Close,total_earn_money,day_earn_percent,total_day_earn_percent
0,0.533754,0.543659,0.548314,0.536711,-11.695505,-2.653413,66.666667,-0.005697,48.673294,-0.002661,...,-66.666667,44.618897,-0.096734,0.000299,17295.0,17320.84,17104.098245,10000.000000,0.000000,0.000000
1,0.536746,0.531401,0.474294,0.507837,-13.616024,-28.930653,-66.666667,-0.022596,35.534674,-0.021143,...,-66.666667,43.785182,-0.120874,-0.000325,16904.2,17303.52,17021.189058,9538.077479,-4.842931,-4.619225
2,0.507831,0.495206,0.492065,0.486015,-13.741791,-31.016597,-66.666667,-0.002331,34.491701,-0.019251,...,-66.666667,43.702440,-0.122264,-0.002735,16864.8,17254.79,16878.906184,9493.615126,-0.468340,-5.063849
3,0.486008,0.485161,0.475934,0.489673,-13.854256,-33.061095,-66.666667,-0.002188,33.469452,-0.017579,...,-66.666667,43.624845,-0.123484,-0.003244,16827.9,17211.45,16701.494010,9452.071272,-0.439521,-5.479287
4,0.489666,0.483385,0.491256,0.485094,-13.412647,-25.079920,-66.666667,0.004249,37.460040,-0.010860,...,-66.666667,43.819102,-0.117549,-0.003413,16899.4,17161.66,16809.886503,9532.393011,0.842619,-4.676070
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2928,0.581233,0.584890,0.584782,0.584447,4.649710,60.193910,66.666667,0.003522,80.096955,0.015180,...,66.666667,51.464350,0.048091,0.004395,22451.6,22231.53,22234.568580,5317.042517,0.699499,-46.829575
2929,0.584436,0.548963,0.578829,0.546383,3.412908,31.892320,66.666667,-0.011861,65.946160,0.002748,...,66.666667,50.613423,0.036300,0.003947,22185.3,22280.62,22571.933251,5437.857102,2.221732,-45.621429
2930,0.546378,0.523817,0.530940,0.518224,2.359017,12.822337,-66.666667,-0.010336,56.411168,-0.006273,...,66.666667,49.899435,0.025975,0.002502,21956.0,22277.60,22543.726016,5550.264925,2.025269,-44.497351
2931,0.518229,0.496928,0.517094,0.502394,3.096597,21.461407,-66.666667,0.007151,60.730704,0.000713,...,66.666667,50.381100,0.032563,0.000580,22113.0,22245.23,22565.951443,5470.888755,-1.450883,-45.291112


In [23]:
np.sign(-0.53)

-1.0

In [20]:
(16904.2 - 17295) / 17295

-0.022596126047990706

In [16]:
input_df[['Close','pred_Close','trading','total_earn_money','day_earn_percent','total_day_earn_percent']]

KeyError: "['trading'] not in index"